# 基于内容的协同过滤的推荐系统实现

代码依赖：
1. pandas，用来读取 csv 数据
2. ml-latest-small 数据集，解压缩后和该文件放在同一目录下。

In [1]:
import pandas as pd
from math import *

## 读取数据

我们用到的数据是 movies.csv，包含电影的 title、movieId 等信息。ratings.csv 包含了 userId、用户对电影的评分。下面代码通过 pandas 来读取 csv 文件，并输出前五行查看结果

In [2]:
# ratings.csv文件中包含了完整数据集。
data_path='./ml-latest-small/ratings.csv'
# 读取数据 
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print(movies.head(5))
print(ratings.head(5))

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [3]:
# 通过 movieId 进行一个表连接
# 输出到 data.csv 中，方便后面直接使用
data = pd.merge(movies, ratings, on='movieId')
data[['userId', 'rating', 'movieId', 'title']].sort_values('userId').to_csv('./ml-latest-small/data.csv', index=False)
# 采用Python字典来表示每位用户评论的电影和评分
file = open('./ml-latest-small/data.csv', 'r', encoding='UTF-8')

# 读取data.csv中每行中除了名字的数据
# 存放每位用户评论的电影和评分
data = {} 
for line in file.readlines()[1:]:
    # 有些 title 也有 "," 因此我们要判断一下长度
    line = line.strip().split(',') # 去掉每行头尾空白
    if len(line) > 4:
        for i in range(4, len(line)):
            line[3] = line[3] + ',' + line[i]
        line = line[0: 4]
    # 如果字典中没有某位用户，则使用用户ID来创建这位用户
    if not line[0] in data.keys():
        data[line[0]] = {line[3]: line[1]}
    #  否则直接添加以该用户ID为key字典中
    else:
        data[line[0]][line[3]] = line[1]

file.close()

In [4]:
# 计算两用户之间的Pearson相关系数
def pearson_sim(user1, user2):
    # 取出两位用户评论过的电影和评分
    user1_data = data[user1]
    user2_data = data[user2]
    distance = 0
    common = {}
 
    # 找到两位用户都评论过的电影
    for key in user1_data.keys():
        if key in user2_data.keys():
            common[key] = 1
    if len(common) == 0:
        return 0 # 如果没有共同评论过的电影，则返回0
    n = len(common) # 共同电影数目
 
    ##计算评分和
    sum1 = sum([float(user1_data[movie]) for movie in common])
    sum2 = sum([float(user2_data[movie]) for movie in common])
 
    ##计算评分平方和
    sum1Sq = sum([pow(float(user1_data[movie]), 2) for movie in common])
    sum2Sq = sum([pow(float(user2_data[movie]), 2) for movie in common])
 
    ##计算乘积和
    p_sum = sum([float(user1_data[it])*float(user2_data[it]) for it in common])
 
    ##计算相关系数
    num = p_sum - (sum1 * sum2 / n)
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    return num / den

In [14]:
# 计算某个用户与其他用户的相似度
# 我们通过定义一个阈值 0.75 来判断两个用户是十分相似的（强相关）
def get_similar_user(target_user):
    res = []
    for user_id in data.keys():
        # 排除与自己计算相似度
        if not user_id == target_user:
            similar_rate = pearson_sim(target_user, user_id)
            if similar_rate > 0.75:
                res.append((user_id, similar_rate)) 
    res.sort(key=lambda val:val[1], reverse=True) # 按照元组中第二个元素进行排序，默认从小到大，降序需要使用reverse=True
    return res

In [15]:
# 根据相似度来推荐用户：
# 这里我们可以对达到阈值的用户都做一个推荐
def recommend(user):
    recommendations = []
    for similar_user in get_similar_user(user):
        user_id = similar_user[0]
        items = data[user_id]
        for item in items.keys():
            if item not in data[user].keys():
                recommendations.append((item, items[item]))
    # 按照评分来进行排序
    recommendations.sort(key=lambda val:val[1], reverse=True)
    # 返回评分最高的10部电影
    return recommendations[:10]

In [16]:
# 获得用户有多少
map_data = pd.read_csv('./ml-latest-small/movies.csv')
user_data = pd.read_csv('./ml-latest-small/data.csv')
user_num = user_data.userId.unique().shape[0]
print("user num: " + str(user_num))

user num: 610


In [17]:
def get_movie_id(movie):
    for i in range(len(map_data)):
        if str(map_data['title'][i]) == movie:
            return map_data['movieId'][i]

In [18]:
# 输出到文件中
user_list = []
movies_list = []

for i in range(1, user_num + 1):
    recommend_list = recommend(str(i))
    for movie in recommend_list:
        # 这里的 movie[0] 可能有 "" 要处理
        temp_movie = movie[0]
        if movie[0].startswith('"') and movie[0].endswith('"'):
            temp_movie = movie[0][1: -1]
        user_list.append(i)
        movie_id = get_movie_id(temp_movie)
        if movie_id == None:
            print(movie[0])
            print(temp_movie)
        movies_list.append(-1 if movie_id == None else movie_id)

# 字典中的key值即为csv中列名
dataframe = pd.DataFrame({'userId': user_list, 'movieId': movies_list})

# 将DataFrame存储为csv,index表示是否显示行名，default=True
dataframe.to_csv("./movie.csv", index=False, sep=',')

In [ ]:
# 结果评估